In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df = pd.read_excel("C:/Users/EzgiOzgen/Downloads/Yeni klasör (2)/merc.xlsx")
df


,year,price,transmission,mileage,tax,mpg,engineSize
0,2005,5200,Automatic,63000,325,32.1,1.8
1,2017,34948,Automatic,27000,20,61.4,2.1
2,2016,49948,Automatic,6200,555,28.0,5.5
3,2016,61948,Automatic,16000,325,30.4,4.0
4,2016,73948,Automatic,4000,325,30.1,4.0
...,...,...,...,...,...,...,...
13114,2020,35999,Automatic,500,145,55.4,2.0
13115,2020,24699,Automatic,2500,145,55.4,2.0
13116,2019,30999,Automatic,11612,145,41.5,2.1
13117,2019,37990,Automatic,2426,145,45.6,2.0


In [4]:
#Checking number of unique rows in each feature

df.nunique().sort_values()

transmission       4
year              25
engineSize        26
tax               40
mpg              120
price           4424
mileage         9716
dtype: int64

In [5]:
#Checking number of unique rows in each feature
target = 'price'
features = [i for i in df.columns if i not in [target]]
original_df = df.copy(deep=True)
nu = df[features].nunique().sort_values()
nf = []; cf = []; nnf = 0; ncf = 0; #numerical & categorical features

for i in range(df[features].shape[1]):
    if nu.values[i]<=20:cf.append(nu.index[i])
    else: nf.append(nu.index[i])

print('\n\033[1mInference:\033[0m The Datset has {} numerical & {} categorical features.'.format(len(nf),len(cf)))


Inference: The Datset has 5 numerical & 1 categorical features.


In [6]:
#Removal of any Duplicate rows (if any)

counter = 0
rs,cs = original_df.shape

df.drop_duplicates(inplace=True)

if df.shape==(rs,cs):
    print('\n\033[1mInference:\033[0m The dataset doesn\'t have any duplicates')
else:
    print(f'\n\033[1mInference:\033[0m Number of duplicates dropped/fixed ---> {rs-df.shape[0]}')


Inference: Number of duplicates dropped/fixed ---> 259


In [7]:
#Check for empty elements

nvc = pd.DataFrame(df.isnull().sum().sort_values(), columns=['Total Null Values'])
nvc['Percentage'] = round(nvc['Total Null Values']/df.shape[0],3)*100
print(nvc)

              Total Null Values  Percentage
year                          0         0.0
price                         0         0.0
transmission                  0         0.0
mileage                       0         0.0
tax                           0         0.0
mpg                           0         0.0
engineSize                    0         0.0


In [8]:
#Converting categorical Columns to Numeric

df3 = df.copy()

ecc = nvc[nvc['Percentage']!=0].index.values
fcc = [i for i in cf if i not in ecc]
#One-Hot Binay Encoding
oh=True
dm=True
for i in fcc:
    #print(i)
    if df3[i].nunique()==2:
        if oh==True: print("\033[1mOne-Hot Encoding on features:\033[0m")
        print(i);oh=False
        df3[i]=pd.get_dummies(df3[i], drop_first=True, prefix=str(i))
    if (df3[i].nunique()>2):
        if dm==True: print("\n\033[1mDummy Encoding on features:\033[0m")
        print(i);dm=False
        df3 = pd.concat([df3.drop([i], axis=1), pd.DataFrame(pd.get_dummies(df3[i], drop_first=True, prefix=str(i)))],axis=1)
        
df3.shape


Dummy Encoding on features:
transmission


(12860, 9)

In [24]:
df1 = df3.copy()

#features1 = [i for i in features if i not in ['CHAS','RAD']]
features1 = nf

for i in features1:
    Q1 = df1[i].quantile(0.25)
    Q3 = df1[i].quantile(0.75)
    IQR = Q3 - Q1
    df1 = df1[df1[i] <= (Q3+(1.5*IQR))]
    df1 = df1[df1[i] >= (Q1-(1.5*IQR))]
    df1 = df1.reset_index(drop=True)
display(df1.head())
print('\n\033[1mInference:\033[0m\nBefore removal of outliers, The dataset had {} samples.'.format(df3.shape[0]))
print('After removal of outliers, The dataset now has {} samples.'.format(df1.shape[0]))

,year,price,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Other,transmission_Semi-Auto
0,2018,30948,16000,145,47.9,2.1,0,0,0
1,2020,32980,606,145,35.8,2.0,0,0,0
2,2020,26980,1000,145,62.8,1.5,0,0,0
3,2019,23750,278,145,55.4,2.0,0,0,0
4,2016,18980,41713,125,56.6,2.1,0,0,0



Inference:
Before removal of outliers, The dataset had 12860 samples.
After removal of outliers, The dataset now has 7234 samples.


In [73]:
#Splitting the data intro training & testing sets
from sklearn.model_selection import train_test_split
m=[]
for i in df1.columns.values:
    m.append(i.replace('-','_'))
  
    
df1.columns = m
X = df1.drop([target],axis=1)
Y =np.log(df1[target])
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=100)
Train_X.reset_index(drop=True,inplace=True)

print('Original set  ---> ',X.shape,Y.shape,'\nTraining set  ---> ',Train_X.shape,Train_Y.shape,'\nTesting set   ---> ', Test_X.shape,'', Test_Y.shape)

Original set  --->  (7234, 8) (7234,) 
Training set  --->  (5787, 8) (5787,) 
Testing set   --->  (1447, 8)  (1447,)


In [74]:
#Feature Scaling (Standardization)
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

print('\033[1mStandardardization on Training set'.center(120))
Train_X_std = std.fit_transform(Train_X)
Train_X_std = pd.DataFrame(Train_X_std, columns=X.columns)
display(Train_X_std.describe())

print('\n','\033[1mStandardardization on Testing set'.center(120))
Test_X_std = std.transform(Test_X)
Test_X_std = pd.DataFrame(Test_X_std, columns=X.columns)
display(Test_X_std.describe())

                                         Standardardization on Training set                                         


,year,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Other,transmission_Semi_Auto
count,5.787000e+03,5.787000e+03,5.787000e+03,5.787000e+03,5.787000e+03,5.787000e+03,5.787000e+03,5.787000e+03
mean,6.764496e-14,-2.150613e-17,5.891549e-16,-6.017881e-16,1.284306e-15,-5.053270e-16,-1.507078e-16,-4.944301e-16
std,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00
min,-4.199279e+00,-1.220377e+00,-4.067586e+00,-2.733277e+00,-1.804623e+00,-2.868608e-01,-1.314652e-02,-1.167478e+00
25%,-6.956418e-01,-8.152618e-01,2.472961e-01,-8.484263e-01,-1.366066e+00,-2.868608e-01,-1.314652e-02,-1.167478e+00
50%,7.058131e-01,-2.280518e-01,2.472961e-01,1.041326e-01,3.881611e-01,-2.868608e-01,-1.314652e-02,8.565471e-01
75%,7.058131e-01,5.518695e-01,2.472961e-01,7.425497e-01,8.267179e-01,-2.868608e-01,-1.314652e-02,8.565471e-01
max,1.406541e+00,2.995931e+00,3.123884e+00,2.556465e+00,1.703832e+00,3.486011e+00,7.606576e+01,8.565471e-01



                                          Standardardization on Testing set                                          


,year,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Other,transmission_Semi_Auto
count,1447.000000,1447.000000,1447.000000,1447.000000,1447.000000,1447.000000,1.447000e+03,1447.000000
mean,0.003149,-0.002429,0.000788,-0.011574,-0.013723,0.010380,-1.314652e-02,0.003295
std,1.006278,0.997601,0.997853,1.012824,1.004951,1.016766,5.553034e-17,0.999828
min,-4.199279,-1.220377,-2.629292,-2.733277,-1.804623,-0.286861,-1.314652e-02,-1.167478
25%,-0.695642,-0.810764,0.247296,-0.929495,-1.366066,-0.286861,-1.314652e-02,-1.167478
50%,0.705813,-0.219381,0.247296,0.104133,0.388161,-0.286861,-1.314652e-02,0.856547
75%,0.705813,0.526464,0.247296,0.742550,0.826718,-0.286861,-1.314652e-02,0.856547
max,1.406541,2.995931,3.123884,2.556465,1.265275,3.486011,-1.314652e-02,0.856547


In [75]:
#Testing a Linear Regression model with statsmodels
from statsmodels.formula import api
Train_xy = pd.concat([Train_X_std,Train_Y.reset_index(drop=True)],axis=1)
a = Train_xy.columns.values

API = api.ols(formula='{} ~ {}'.format(target,' + '.join(i for i in Train_X.columns)), data=Train_xy).fit()
#print(API.conf_int())
#print(API.pvalues)
API.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     2067.
Date:                Sat, 27 Aug 2022   Prob (F-statistic):               0.00
Time:                        13:08:52   Log-Likelihood:                 2847.1
No. Observations:                5787   AIC:                            -5676.
Df Residuals:                    5778   BIC:                            -5616.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 10.0726      0.002   5175.237      0.000      10.069      10.076
year                       0.1471      0.003     46.081      0.000       0.141       0.153
mileage                   -0.0702      0.003    -22.617      0.000      -0.076      -0.064
tax                       -0.0068      0.002     -3.092      0.002      -0.011      -0.002
mpg                       -0.0811      0.002    -40.448      0.000      -0.085      -0.077
engineSize                 0.0907      0.002     43.789      0.000       0.087       0.095
transmission_Manual       -0.0479      0.002    -22.145      0.000      -0.052      -0.044
transmission_Other        -0.0029      0.002     -1.510      0.131      -0.007       0.001
transmission_Semi_Auto     0.0048      0.002      2.316      0.021       0.001       0.009
==============================================================================
Omnibus:                       38.424   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.785
Skew:                           0.171   Prob(JB):                     1.39e-09
Kurtosis:                       3.227   Cond. No.                         3.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [84]:
#Linear Regression



from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures


MLR = LinearRegression().fit(Train_X_std,Train_Y)
pred1 = MLR.predict(Train_X_std)
pred2 = MLR.predict(Test_X_std)

print('{}{}\033[1m Evaluating Multiple Linear Regression Model \033[0m{}{}\n'.format('<'*3,'-'*35 ,'-'*35,'>'*3))
#print('The Coeffecient of the Regresion Model was found to be ',MLR.coef_)
print('The Intercept of the Regresion Model was found to be ',MLR.intercept_)


print(((pred1.reshape((len(pred1),1)))))
print(Test_Y.values.reshape((len(Test_Y),1)))



<<<----------------------------------- Evaluating Multiple Linear Regression Model ----------------------------------->>>

The Intercept of the Regresion Model was found to be  10.0726144970099
[[ 9.7774773 ]
 [10.14323827]
 [10.355787  ]
 ...
 [10.17420086]
 [ 9.53610934]
 [10.16370892]]
[[ 9.92573816]
 [10.16581335]
 [ 9.6181361 ]
 ...
 [10.19238139]
 [ 9.82547196]
 [10.46253175]]


In [80]:
#Creating a Ridge Regression model

RLR = Ridge().fit(Train_X_std,Train_Y)
pred1 = RLR.predict(Train_X_std)
pred2 = RLR.predict(Test_X_std)

print('{}{}\033[1m Evaluating Ridge Regression Model \033[0m{}{}\n'.format('<'*3,'-'*35 ,'-'*35,'>'*3))
#print('The Coeffecient of the Regresion Model was found to be ',MLR.coef_)
print('The Intercept of the Regresion Model was found to be ',MLR.intercept_)
RLR.score(Train_X_std,Train_Y)
print(((pred1.reshape((len(pred1),1)))))
print(Test_Y.values.reshape((len(Test_Y),1)))

<<<----------------------------------- Evaluating Ridge Regression Model ----------------------------------->>>

The Intercept of the Regresion Model was found to be  10.0726144970099
[[ 9.77748242]
 [10.14322711]
 [10.35573425]
 ...
 [10.17420653]
 [ 9.53617449]
 [10.16370713]]
[[ 9.92573816]
 [10.16581335]
 [ 9.6181361 ]
 ...
 [10.19238139]
 [ 9.82547196]
 [10.46253175]]


In [81]:
#Creating a Ridge Regression model

LLR = Lasso().fit(Train_X_std,Train_Y)
pred1 = LLR.predict(Train_X_std)
pred2 = LLR.predict(Test_X_std)

print('{}{}\033[1m Evaluating Lasso Regression Model \033[0m{}{}\n'.format('<'*3,'-'*35 ,'-'*35,'>'*3))
#print('The Coeffecient of the Regresion Model was found to be ',MLR.coef_)
print('The Intercept of the Regresion Model was found to be ',MLR.intercept_)
print(LLR.score(Train_X_std,Train_Y))
print(((pred1.reshape((len(pred1),1)))))
print(Test_Y.values.reshape((len(Test_Y),1)))

<<<----------------------------------- Evaluating Lasso Regression Model ----------------------------------->>>

The Intercept of the Regresion Model was found to be  10.0726144970099
0.0
[[10.0726145]
 [10.0726145]
 [10.0726145]
 ...
 [10.0726145]
 [10.0726145]
 [10.0726145]]
[[ 9.92573816]
 [10.16581335]
 [ 9.6181361 ]
 ...
 [10.19238139]
 [ 9.82547196]
 [10.46253175]]


In [82]:
#Creating a ElasticNet Regression model

ENR = ElasticNet().fit(Train_X_std,Train_Y)
pred1 = ENR.predict(Train_X_std)
pred2 = ENR.predict(Test_X_std)

print('{}{}\033[1m Evaluating Elastic-Net Regression Model \033[0m{}{}\n'.format('<'*3,'-'*35 ,'-'*35,'>'*3))
#print('The Coeffecient of the Regresion Model was found to be ',MLR.coef_)
print('The Intercept of the Regresion Model was found to be ',MLR.intercept_)
print(ENR.score(Train_X_std,Train_Y))
print(((pred1.reshape((len(pred1),1)))))
print(Test_Y.values.reshape((len(Test_Y),1)))

<<<----------------------------------- Evaluating Elastic-Net Regression Model ----------------------------------->>>

The Intercept of the Regresion Model was found to be  10.0726144970099
0.0
[[10.0726145]
 [10.0726145]
 [10.0726145]
 ...
 [10.0726145]
 [10.0726145]
 [10.0726145]]
[[ 9.92573816]
 [10.16581335]
 [ 9.6181361 ]
 ...
 [10.19238139]
 [ 9.82547196]
 [10.46253175]]
